## **Basics Scraping With Beautifulsoup4**

In [ ]:
import requests
from bs4 import BeautifulSoup

### **What we need to know about Beautifulsoup4 (Bs4)?**

Bs4 merupakan librarry yang digunakan untuk melakukan web scraping pada suatu website. Dalam penggunaan Bs4 kita bergantung pada *Request* dari suatu website untuk mendapatkan data yang kita mau. Meskipun terlihat simpel Bs4 merupakan web scraper yang juga membutuhkan *parser* seperti **html parser** atau **xml parser** untuk mengekstrak data dari suatu website. 

### **Scraping Method**

Biasanya dalam scraping menggunakan Bs4 kita membutuhkan:

    1. Inisiasi URL
    2. Buat Request
    3. Parsing Data
    4. Ekstraksi Data


In [ ]:
# membuat insisiaisi website
website = 'https://subslikescript.com/movie/The_Love_Club-19896920'
# melakukan request ke website
result = requests.get(website)
# menggambil konten dari hasil request
content = result.text

# membuat soup untuk parsing lxml/html
soup = BeautifulSoup(content, 'lxml')
# memuncuklan hasil parsing
# print(soup.prettify())

**Screenshot website yang ingin dilakukan scraping**

![media - website subslikescript](../media/basics_scraping_subslikescript.png)

Untuk melihat elemen — kita bisa melakukan inspect langsung pada website dan pilih elemen yang ingin diambil datanya. Pada contoh kali ini saya ingin mengambil data judul dari film "THE LOVE CLUB (2023) - FULL TRANSCRIPT".

In [ ]:
# karena judul film berada di dalam tag <h1> dan <article>
# kita perlu mencari tag sebelumnya yaitu <article>
box = soup.find('article', class_='main-article')
# mengambil judul film dari tag <h1>
title = box.find('h1').text
# print hasil judul film
print(f'Title: {title}')

Title: The Love Club (2023) - full transcript


In [ ]:
# selanjutnya kita akan mengambil transkrip film dan membuat file txtnya

# mengambil transkrip film dari tag <div> dengan class 'full-script'
transcript = box.find('div', class_='full-script').get_text(strip=True, separator=' ')

# membuat file
# ketika membuat file, gunakan mode 'w' untuk menulis
# dan encoding 'utf-8' untuk mendukung karakter non-ASCII
# pastikan direktori '../data export/' sudah ada
# jika belum ada, buat direktori tersebut terlebih dahulu
# dalah contoh ini, kita akan menyimpan file dengan nama sesuai judul film
with open(f'../data export/{title}.txt', 'w', encoding= 'utf-8') as file:
    file.write(transcript)

### **Scraping Multiple Element**

![subslikescript - multiple element href true](../media/href_true.png)


![subslikescript - href zoom](../media/href_zoom.png)

Dalam contoh kali ini, untuk mengambil beberapa element yang sama kita perlu memahami struktur html dari suatu website. Jika diperhatikan **href** menunjukkan bahwa setiap movies akan menampilkan tampilan transcript dihalaman yang terpisah

In [ ]:
# membuat insisiaisi website
website = 'https://subslikescript.com/movies'
# melakukan request ke website
result = requests.get(website)
# menggambil konten dari hasil request
content = result.text

# membuat soup untuk parsing lxml/html
soup = BeautifulSoup(content, 'lxml')

box = soup.find('article', class_='main-article')

links = []

for link in box.find_all('a', href=True):
    # mengambil link dari atribut href
    links.append(link['href'])
print(links)

['/movie/Gereza-22857380', '/movie/Dragon_Hunter-22008250', '/movie/House_of_Inequity-5598934', '/movie/Jackpot_Island_Kumanthong_Returns-25260658', '/movie/Match-13490000', '/movie/Snow_White_and_the_Fairytale_Fun_Force-22700808', '/movie/The_Love_Club-19896920', '/movie/Haunted_Hotties-24018636', '/movie/One_Year_Off-14130940', '/movie/Sweeter_Than_Chocolate-25391002', '/movie/Disquiet-25869142', '/movie/Among_the_Beasts-26343318', '/movie/Taming_Speed-25969676', '/movie/Randy_Feltface_The_Last_Temptation_of_Randy-17527566', '/movie/Alemanji-23666980', '/movie/Darkheart_Manor-12569866', '/movie/Innocent_Vengeance-26505553', '/movie/Ho_Ja_Mukt-25150506', '/movie/Boy_from_Nowhere-26713915', '/movie/Rent-a-Groom-15352290', '/movie/Unlocked-26160190', '/movie/j-hope_IN_THE_BOX-26425683', '/movie/Suki-26734145', '/movie/Re-Resonator_Looking_Back_at_from_Beyond-26625455', '/movie/Americas_National_Parks_at_100-26740893', '/movie/Under_His_Influence-26083016', '/movie/Why_Cant_My_Life_Be_a_

In [ ]:
root = 'https://subslikescript.com'
# mengambil setiap link dan mengambil transkrip filmnya menggunakan loop
for link in links:
    movies = f'{root}{link}'
    # melakukan request ke website
    result = requests.get(movies)
    # menggambil konten dari hasil request
    content = result.text
    # membuat soup untuk parsing lxml/html
    soup = BeautifulSoup(content, 'lxml')

    box = soup.find('article', class_='main-article')
    # mengambil judul film dari tag <h1>
    title = box.find('h1').text
    # mengambil transkrip film dari tag <div> dengan class 'full-script'
    transcript = box.find('div', class_='full-script').get_text(strip=True, separator=' ')

    with open(f'../data export/{title}.txt', 'w', encoding= 'utf-8') as file:
        file.write(transcript)
        
# menampilkan pesan selesai
print('Semua transkrip film telah diambil dan disimpan!')

OSError: [Errno 22] Invalid argument: "../data export/Why Can't My Life Be a Rom Com? (2023) - full transcript.txt"

Jika kita lihat diatas terdapat error yang disebabkan karena adanya karakter ilegal untuk membuat nama file. Kita bisa menggunakan try-except untuk solusi seperti ini atau gunakaan fungsi regex untuk membersihkan karakter yang tidak aman, tetapi fokus utama dalam mengambil multiple element sudah berhasil diterapkan. jika kita cek di directory file sudah berhasil dibuat dan diambil datanya.

![multiple element scraping](../media/multiple_element_scraping.png)

### **Pagination Using Bs4**

![pagination element](../media/pagination.png)

Untuk melakukan pagination kita perlu mangambil elemen yang memuat akses ke halama berikutnya. Dalam website "subslikescript.com" dapan dilihat elemen berapa pada box ul dan link pada li. Jika diperhatikan juga situs akan berubah menjadi "subslikescript.com/**movies?page=2**" artinya setiap perubahan ke halaman berikutnya situs akan berubah menjadi seperti ini. Jika sudah diketahui karakteristik dari web ketika berpindah ke halaman berikutnya kita bisa mengakali dengan mengambil banyak halaman dan memasukkannya dengan angka pada **page={}**.

Sayangnya website ini telah membatasi aktivitas seperti ini, karena sistem website akan mengenali pergerakan tidak wajar dari user. Walaupun mungkin bisa diatasi menggunakan fungsi time untuk melakukan jeda pada proses scraping, akan tetapi tidak ada jaminan data dapat terambil semua.

In [ ]:
import re

def clean_text(text):
    return re.sub(r'[\\/*?:"<>|]', "", text)

In [ ]:
import time

In [ ]:
root = 'https://subslikescript.com'
base_url = 'https://subslikescript.com/movies'
result = requests.get(base_url)
# menggambil konten dari hasil request
content = result.text

# membuat soup untuk parsing lxml/html
soup = BeautifulSoup(content, 'lxml')

# pagination adalah elemen yang berisi link ke halaman-halaman berikutnya
pagination = soup.find('ul', class_='pagination')
pages = pagination.find_all('li', class_='page-item')
last_pages = pages[-2].text  # mengambil jumlah halaman terakhir


# for page in range(1, int(last_pages) + 1)[:2]: # [:2] mengambil 2 halaman pertama
for page in range(1, 3): 
    try:
        get_ref = requests.get(f'{base_url}?page={page}')
        pg_content = get_ref.text
        soup = BeautifulSoup(pg_content, 'lxml')

        box = soup.find('article', class_='main-article')
        # mengambil link dari setiap halaman
        # for link in box.find_all('a', href=True):
        # # mengambil link dari atribut href
        #     links.append(link['href'])

        # reset link untuk tiap halaman
        links = [link['href'] for link in box.find_all('a', href=True)]
    
        # mengambil setiap link dan mengambil transkrip filmnya menggunakan loop
        for link in links:
            try:
                # melakukan request ke website menggunakan link yang telah diambil
                result = requests.get(f'{root}{link}')
                time.sleep(2)  # menunggu 2 detik untuk menghindari terlalu banyak request
                # menggambil konten dari hasil request
                content = result.text
                # membuat soup untuk parsing lxml/html
                soup = BeautifulSoup(content, 'lxml')

                box = soup.find('article', class_='main-article')
                # mengambil judul film dari tag <h1>
                title = box.find('h1').text
                # membersihkan judul film dari karakter yang tidak diinginkan
                safe_title = clean_text(title)
                # mengambil transkrip film dari tag <div> dengan class 'full-script'
                transcript = box.find('div', class_='full-script').get_text(strip=True, separator=' ')

                with open(f'../data export/{safe_title}.txt', 'w', encoding= 'utf-8') as file:
                    file.write(transcript)
                print(f'Transkrip untuk {safe_title} telah disimpan.')
            except Exception as e:
                print(f'Error pada link {link}: {e}')
                continue
    except:
        print(f'Error pada halaman {page}, melanjutkan ke halaman berikutnya.')
        continue

Transkrip untuk Gereza (2022) - full transcript telah disimpan.
Transkrip untuk Dragon Hunter (2022) - full transcript telah disimpan.
Transkrip untuk House of Inequity (2021) - full transcript telah disimpan.
Transkrip untuk Jackpot Island Kumanthong Returns (2022) - full transcript telah disimpan.
Transkrip untuk Match (2022) - full transcript telah disimpan.
Transkrip untuk Snow White and the Fairytale Fun Force (2023) - full transcript telah disimpan.
Transkrip untuk The Love Club (2023) - full transcript telah disimpan.
Transkrip untuk Haunted Hotties (2022) - full transcript telah disimpan.
Transkrip untuk One Year Off (2023) - full transcript telah disimpan.
Transkrip untuk Sweeter Than Chocolate (2023) - full transcript telah disimpan.
Transkrip untuk Disquiet (2023) - full transcript telah disimpan.
Transkrip untuk Among the Beasts (2023) - full transcript telah disimpan.
Transkrip untuk Taming Speed (2022) - full transcript telah disimpan.
Transkrip untuk Randy Feltface The L